# 개요
* 참여중인 데이터 엔지니어링 스터디에서 배우는 내용 정리
  * 데이터 수집, 정제 : pyspark, airflow
  * 저장 : elasticsearch
  * 시각화 : kibana

# 1주차 과제리뷰

* SparkContext와 SparkSession 두가지 방법 가능
  * **SparkContext(Low level)**
    * SparkConf(Configuration)에 다양한 설정 가능, SparkContext(conf=)안에 할당
    * `SparkConf().getAll()`을 사용해 옵션에 들어가는 정보들을 확인 가능
  ```python
    from pyspark import SparkConf, SparkContext 
  # https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.SparkContext.html

  spark_conf = SparkConf().setAppName("spark-context-test")
  spark = SparkContext(conf = spark_conf)

  SparkConf().getAll()
  >>> [('spark.master', 'local[*]'),
  ('spark.submit.pyFiles', ''),
  ('spark.submit.deployMode', 'client'),
  ('spark.app.name', 'spark-context-test'),
  ('spark.executor.extraJavaOptions',
    '-Dio.netty.tryReflectionSetAccessible=true'),
  ('spark.ui.showConsoleProgress', 'true'),
  ('spark.driver.extraJavaOptions',
    '-Dio.netty.tryReflectionSetAccessible=true')]
  ```
  * **SparkSession(High level, SparkContext 감싸고 있는, 래핑되어있는 버전)**
    * master와 appName은 항상 넣는 것이 좋음
      * master : "local" 또는 spark://spark-master:7077
  ```python
  from pyspark.sql import SparkSession

  spark = (
      SparkSession.builder
          .master("local") # 또는 spark://spark-master:7077로도 넣을 수 있음
          .appName("spark-session-test")
  #         .config("spark.some.config.option", "some-value")
          .getOrCreate()
  )
  print(f"Hello, {spark.sparkContext.sparkUser()}")

  # enableHiveSupport()
  ```
<br>

* Read Data
  * 데이터를 읽는 방법들
    * **spark.read.json**
    ```python
    df = spark.read.json("../data/2024-05-19-14.json")
    ```
    * **spark.read.format(포맷 지정해서 읽기 가능)**
    ```python
    df = spark.read.format('org.apache.spark.sql.json') \
        .load("../data/2024-05-19-14.json")
    ```
    * **spark.read.schema(schema).json(정의해둔 Schema로 읽기 가능)**
    ```python
    df = spark.read.schema(schema).json("../data/2024-05-19-14.json")
    ```
  * 데이터를 읽는 방법(RDD)
    * **sc.textFile('../data/2024-05-19-14.json')**
      * 데이터 포맷에 관계없이 텍스트로 읽을 수 있으며, RDD타입으로 읽어짐
      * 이미지, 음성, stream data 등은 RDD타입이 더 적합할 수 있음
      ```python
      rdd = sc.textFile('../data/2024-05-19-14.json')
      rdd.take(1)
      ```
  * 읽은 데이터를 확인하기
    * printSchema()로 데이터의 타입, nullable 등을 확인
    ```python
    df = spark.read.json("../data/2024-05-19-14.json")

    df.printSchema()
    >>> root
    |-- actor: struct (nullable = true)
    |    |-- avatar_url: string (nullable = true)
    |-- created_at: string (nullable = true)
    ```
    * df.count() : row 수
    * df.show(1, False) : row 수, truncate여부 지정하여 확인 가능
    * df.dtypes : 데이터 타입
    * df.select('type') : type필드만 가져오기
    * df.select('type').distinct() : type필드 중 unique값만 가져오기
    * df.select('type').distinct().collect() : 가져온 데이터를 파이썬 타입으로 변경
<br>

* Schema 정의
  * pyspark.sql.types에 정의된 타입 사용
  ```python
  from pyspark.sql.types import StructType, StructField, StringType, DateType, LongType, BooleanType
  ```
  * StructType으로 Schema 정의 가능
  ```python
  schema = StructType([
      # StructField('필드이름(컬럼명)', 타입, nullable)
      StructField("actor", StringType(), True),
      StructField("created_at", DateType(), True), # 시간데이터지만 string이었던 것을 DataType으로 typecase
      StructField("id", LongType(), False),
      StructField("payload", StringType(), True),
  ])
  ```
    * 참고 : 위에 사용된 gh archive 데이터의 payload는 실제 데이터가 json으로 들어있어(Nested json),
      실제로는 StructType으로 정의 후, Nested Schema를 직접 정의해주어야 함
  * 정의해 둔 Schema로 데이터를 읽는 경우, 기존과 다른 Schema로 출력됨을 확인 가능
  ```python
  df = spark.read.schema(schema).json("../data/2024-05-19-14.json")
  df.printSchema()
  >>> root
  |-- actor: string (nullable = true)
  |-- created_at: date (nullable = true) # 위에 올려보면 처음엔 string이었음
  |-- id: long (nullable = true)
  |-- payload: string (nullable = true)
  ```
<br>

* Write Data
  * df.write : 아래와 같이 폴더에 저장 가능
  ```python
  df2 = df.filter(df.type == 'CommitCommentEvent')
  df2.write \
    .option("compression", "gzip") \
    .mode('Overwrite') \
    .json("../data/output.json")
  ```
  * 저장한 데이터는 아래와 같이 읽어볼 수 있음
  ```python
  df3 = spark.read.schema(schema).json("../data/output.json")
  df3.printSchema()
  >>> root
  |-- actor: string (nullable = true)
  |-- created_at: date (nullable = true)
  |-- id: long (nullable = true)
  |-- org: string (nullable = true)
  |-- payload: string (nullable = true)
  |-- public: boolean (nullable = true)
  |-- repo: string (nullable = true)
  |-- type: string (nullable = true)
  ```

# 2주차 수업정리

## RDD

* RDD (Resilient Distributed Dataset, 분산 데이터 모델)
  * RDD를 사용할 때는 How를 기술
  * Default는 여러 클러스터의 메모리 상에 존재하는 데이터 구조
  * MapReduce와 비슷한 역할
  * **데이터 포맷이 텍스트가 아니어도 전부 텍스트로 읽어오는데, 이럴 때 RDD를 사용**
  * 파이썬에서 RDD를 생성하는 예제
    * 보통 대용량 처리에 쓰기 때문에, 이처럼 Local에서 처리하는 일은 실제로 많지 않음을 참고
    ```python
    #  local dataset
    data = ["one", "two", "three"] 
    sc.parallelize(data)
    # range
    sc.range(1, 6) 
    # read file
    sc.textFile("data/data1.txt")
    ```
<br>

* (RDD데이터 정제시에 사용하는)RDD API에 들어있는 Function은 2가지가 있음
  * RDD Transformation & RDD Actions
<br>

* RDD Transformation
  * **RDD를 또 다른 RDD로 변환하는 것(return value가 RDD인 Operation)**
    * 기존의 변수는 바뀌지 않고, 새로운 RDD를 리턴한다
      * 예를 들어 sc.range(1, 6)로 만든 RDD의 모든 숫자에 1을 더한다면, 기존 변수는 건드리지 않고 새로운 RDD생성
      * 기존 변수를 건드리지 않고 새로운 값을 리턴받아 사용해야한다는 점은 함수형 프로그래밍의 기본사항임을 참고
  * **기존 데이터는 메모리에 남아있으며, Spark가 메모리 삭제 해주기 전까지는 남아있음**
  * RDD Transformation예시
    * 문장을 단어로 split하거나 문단을 문장으로 split해서 리턴하는 경우
    ```python
    rdd.flatMap(lambda line: line.split(‘ ‘))
    ```
    * 모든 숫자 중 3보다 큰 것만 가져오는 filtering (3보다 큰 것만 추출한 새로운 데이터 구조 리턴)
    ```python
    rdd.filter(lambda x: x > 3)
    ```
<br>

* RDD Transformation with shuffle
  * (대용량 데이터가 분산 처리되고 있고, 여러 노드에 데이터가 저장되어 있는 상황에서) 각 Node에서 데이터 이동이 이루어짐
    * 예를 들어 10개 Cluster가 10%씩 데이터가 있을 때, Node에서 **데이터 이동이 이루어져야 Count되는 경우가 Shuffle**
    * **Disk I/O와 Network Traffic(I/O)을 동반하므로, 느리고 부하를 준다**
  * **Shuffle이 들어간 Transformation은 정해져있으므로**, 느릴 수 있다는 것을 미리 생각하고 돌려야함
    * reduceByKey : key가 동일한 value를 가져와서 작업하는데, 같은 Cluster에 있다는 보장은 없음
      * 실습환경인 Local에서는 체감이 없겠지만, 대용량/분산 환경에서는 부하와 함께 메모리가 터지기도 함
      * 기본적으로 계산에 사용하는 것만 메모리에 올려있으므로, 작업을 위해 올리는 과정에서부터 문제될 수 있음
<br>

* RDD Actions
  * RDD를 리턴하는 동작 자체를 의미
    * print로 화면에 보여주거나, write와 같은 동작(사용자에게 반환)
    * **Spark은 lazy하게 동작하게 되어있어, action이 있어야 실질적인 데이터 처리 이루어짐**
      * print나 write없이 transformation만 있는 경우, 동작이 필요없다고 판단
        * 뒤의 다른 동작이 정의될 때까지 기다렸다가 한번에 최적화하게 되어있음
        * 예를 들어 print 등이 없는 경우, 굉장히 빠르게 작업이 끝났다면 계산을 아예 안한 것
  * RDD의 크기가 커서 Driver client로 리턴할 수 없는 경우, 파일로 쓰는 처리만 가능
    * 데이터의 print/write 등은 Driver가 처리하도록 되어있음
      * Executor가 작업 후 넘기면 Driver가 print하는 방식
      * print할 RDD자체가 너무 크면 Driver로 리턴불가하고, Disk I/O만 일어남
  * Action의 종류
    * rdd.saveAsTextFile : 텍스트파일로 저장
    * rdd.collect() : 파이썬 로컬 데이터셋으로 변경
<br>

* RDD API 공식문서 (pyspark.RDD)
  * Transformation이나 Action 등 확인 가능
  * https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.html
  * RDD 관련 기본적인 기능들
    * aggregate, dictinct, filter, foreach, groupby, join
      * join 등은 데이터를 모으는 기능이 있으니 shuffle이 발생함을 유의
  * 분산처리에 특화된 몇가지 API
    * aggregate, aggregateByKey, barrier, cache, cartesian
<br>

* Pair RDD
  * Pair데이터를 다루는 RDD (하나의 Row에 key-value 쌍으로 구성된 경우 등)
    * PairRDD : key-value 쌍으로 구성된 RDD
  * Pair RDD API(InputRDD가 Pair가 아닌 경우 오류)
    * **countByKey** 키 별로 개수 세기
    * **lookup** 단일 키로 값 찾기
    * **mapValues** 키를 변경하지 않고 pairRDD값만 변경가능
      * map과 유사하지만, 키를 변경하지 않음에서 차이
    * **flatMapValues** 각 키 값을 0~1개이상 값으로 매핑해 RDD에 포함한 요소 개수 변경
      * flatmap 예시
        * 원본 : ["Hello world", "Hello, World"]
        * .split(" ") : [["Hello", "world"], ["Hello,", "World"]]
        * flatmap lambda x:x.split : ["Hello", "world", "Hello,", "World"]
    * **reduceByKey** 각 키의 모든 값을, 동일한 타입의 단일 값으로 병합
      * 두 값을 하나로 병합하는 merge 함수 필요 : lambda a, b:a+b
      * 하나의 값이 남을 때까지 merge 함수 호출
    * **foldByKey** : reduceByKey와 같은 기능 (+항등원인 zeroValue를 추가로 전달)
      * a, b:a+b,0 과 같이 입력해, 첫번째 value를 0으로 함
    * **aggregateByKey** : 타입을 바꾸고 싶은 경우 사용 (zeroValue를 받아 RDD값을 병합한다)
      * 타입 변환 함수 : (U, V) → U
      * 병합 함수 : (U, U) → U
      * 함수를 2개 전달하고, 항등원도 전달해 3개의 파라미터를 전달하는 복잡한 API임
<br>

 * Global Variance
   * Spark는 Job을 Clustermanager가 각 노드(클러스터)에 나누어 보내줌
   * 파이썬에서 Global변수를 선언하더라도 노드 안에서만 작용함
   * 모든 클러스터에 대해서 작용할 수 있도록, **BroadCast변수를 사용해야함**
   * broadcast()로 한번 감싸주어야 클러스터 내에서 Global변수로 사용 가능
<br>

* Accumulator
  * `변수 = sc.accumulator('변수이름')`과 같은 형태로 사용
  * 예를 들어 여러 개의 노드가 있을 때, 성공하거나 실패한 횟수를 계산하고자 할때 노드 별로 출력되어 확인 어려움
    * accumulator를 통해 각 노드별 상황을 취합해 하나의 값으로 확인할 수 있음
  * 코드 예시
    * 이해를 위한 예시이며, 실제로는 SparkUI에서 아래 예제와 같은 값은 확인가능하므로 print불필요
    ```python
    ac1 = sc.accumulator("ac1")

    # executor (.add로 값을 더함)
    sc.range(0, 10).foreach(lambda v: ac1.add(v))

    # driver에서 print로 값 출력
    print("Accumulator", ac1.value)
    ```
<br>

* RDD 장점
  * In memory연산과 분산처리(parallel)로 **효율적임**
  * Immutable(기존 변수를 바꾸지 않고 새로운 변수 리턴)하여 기존데이터를 바꾸지 않으므로 **Consistency가 깨지지 않음**
    * 함수형 코딩의 특징에 해당함
  * missing or damaged partition 등으로 인한 node fail 발생시 **알아서 recompute**하는 기능이 있음
<br>

* RDD 사용할 때(어떤 경우에 사용)
  * (low-level transformation와 action을 제공하므로)low-level 연산을 하고싶을 때
  * Unstructured data를 사용할 때 (media streams, text stream, 실시간 데이터셋, image, sound 등)
  * 함수형 프로그래밍에 익숙한 경우(return value가 immutable)
  * 스키마가 불필요한 경우
  * (거의 없지만)  Optimization & perfomance benefit이 불필요한 경우
    * DataFrame이나 Dataset은 훨씬 더 많은 최적화가 적용되어 있고 빠름 
      (table data는 DataFrame 등이 더 좋음)
<br>

* 참고
  * 앞서 실습해본 printSchema()는 스키마와 컬럼을 가졌다는 것이므로, RDD는 대상이 아님
  * 회사에서는 이미지를 처리할 때 RDD를 많이 사용하기도 함(Unstructured data)
  * feature data도 RDD로 읽는게 더 편할 때도 있음 (실제 기업의 모든 데이터가 Strucctured data는 아님)

## DataFrame

* DataFrame : 컬럼으로 organized된 데이터셋 (Spark이므로 distributed)
  * pandas보다 더 다양한 데이터 소스, 분산처리 가능
    * json, parquet, hive,mysql 등
  * 공식문서(pyspark.sql.DataFrame, RDD와 분리된/SQL처럼 처리할 수 있는)
    * 링크 : https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html
  * RDD에서 할 수 있는 것을 DataFrame에서도 대부분 처리 가능 (RDD는 컬럼기능은 불가함은 참고)
<br>

* RDD와 DataFrame의 Transform 방식의 차이 (RDD → DataFrame)
  * map(flatMap) → select
  * Object → Row(columns)
  * Function (lambda base) → Column (SQL)
<br>

* DataFrame의 장점
  * Type-safety보장 : 스키마를 읽었기 때문에 컬럼별 타입 보장
  * Easy to use : 컬럼이 많은 경우 유리 (high-level function, SQL처럼 쿼리하듯이 사용가능)
  * Fast and optimized : Spark의 optimization엔진인 catalyst엔진이 많은 역할을 함
    * 성능비교(Aggregateing 10 million int pairs [Seconds])
      * [RDD인 경우] Python vs Scala : Python 9초 이상 / Scala 4초 이상으로 큰 차이
      * [DataFrame인 경우] Python vs Scala : 모두 2초 이상으로 유의미한 차이가 보이지 않음
        * Dataframe에 optimization이 많이 되어있기 때문
<br>

* DataFrame이 적합한 경우
  * Schema가 있는 Structured data
  * SQL쿼리처럼 High-level transformation이 필요한 경우
  * Type safety를 원하는 경우

## Dataset

* Dataset : DataFrame과 유사하나 타입을 넣어줄 수 있는 Typed API (Dataset[T])
  * DataFrame은 Row타입만 들어갈 수 있는 Dataset과 유사함 (Dataset[Row])

* Dataset 장점 및 특징
  * DataFrame대비 Optimization이 좀 더 반영되어있고, 조금 더 빠름
  * Serialization과 Garbage collection에서 유리
  * Java와 Scala만 지원

* Typed vs Untyped
  * Typed (Java & Scala)
  * Untyped (Python and R) : 대부분의 Dataset API의 이점이 DataFrame API에서도 가능함

* Spark SQL
  * 기존 SQL문처럼 사용가능
  ```python
  ps.sql("SELECT * FROM range(10) where id > 7")
  ```